In [ ]:
# MLP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Load taining data
Data_train = pd.read_csv('mushroom_train.csv')

In [ ]:
# Load test data
Data_test = pd.read_csv('mushroom_test.csv')

In [ ]:
# Separate training data features and labels
X = Data_train.drop(columns='class')
y = Data_train['class']

# Define column transformer
numerical_features = ['cap-diameter', 'stem-height', 'stem-width']
categorical_features = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
                        'gill-attachment', 'gill-spacing', 'gill-color', 'stem-color', 'has-ring',
                        'ring-type', 'habitat', 'season']

column_transformer = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

# Create the pipeline
transformer = Pipeline(steps=[('transformer', column_transformer)])

# Fit and transform training data
X_data = transformer.fit_transform(X)

le = LabelEncoder()
y_data = le.fit_transform(y)

# Split data into training data and validation data
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# Separate test data features and labels
Xtest = Data_test.drop(columns='class')
ytest = Data_test['class']

# Transform test data using the same pipeline
X_test = transformer.transform(Xtest)

y_test = le.transform(ytest)

In [ ]:
# The code is sampled from the EE559 Github page "https://github.com/keithchugg/ee559_spring2023/blob/main/lecture/fmnist_mlp_torch.ipynb"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define MLP model
class MLP(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden):
        super().__init__()
        self.fc1 = nn.Linear(n_features, n_hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Parameters
n_hidden = 128
n_epochs = 40
batch_size = 64
learning_rate = 0.01
l2_reg = 0.001 
n_features = X_data.shape[1]
n_classes = 2
n_splits = 5

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_data.toarray(), dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.long)

# Convert test data to PyTorch tensors
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create a DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define KFold cross-validator
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)


# Training with cross-validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(X_tensor, y_tensor)):
    print(f"Fold {fold + 1}")

    X_train, y_train = X_tensor[train_indices], y_tensor[train_indices]
    X_val, y_val = X_tensor[val_indices], y_tensor[val_indices]

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = MLP(n_features, n_classes, n_hidden).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=l2_reg)

    train_loss_list = []
    val_accuracy_list = []
    f1_list = []
    val_accuracy_list_fold = []
    f1_list_fold = []


    # Training loop
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss_list.append(running_loss / len(train_loader))
        
        model.eval()
        all_predictions = []
        all_labels = []
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device)

                outputs = model(batch_X)
                _, predicted = torch.max(outputs, 1)
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions, average='binary', pos_label=1)
        val_accuracy_list.append(accuracy)
        f1_list.append(f1)

    val_accuracy_list_fold.append(accuracy)
    f1_list_fold.append(f1)
        
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1:.4f}")

print(f"Average accuracy: {np.mean(val_accuracy_list_fold):.4f}")
print(f"Average F1-score: {np.mean(f1_list_fold):.4f}")

plt.figure()
plt.plot(range(1, n_epochs+1), train_loss_list, label="Train Loss")
plt.plot(range(1, n_epochs+1), val_accuracy_list, label="Validation Accuracy")
plt.plot(range(1, n_epochs+1), f1_list, label="Validation F1 Score")
plt.xlabel("Epochs")
plt.ylabel("Loss | Accuracy | F1 Score")
plt.title("Loss | Accuracy | F1 Score vs Epoch on Validation Data")
plt.legend()
plt.show()


In [ ]:
# Evaluate the model on test data
model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(batch_y.cpu().numpy())

# Calculate accuracy and F1 score on test data
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions, average='binary', pos_label=1)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1-score: {test_f1:.4f}")

# Generate the confusion matrix
cm = confusion_matrix(test_labels, test_predictions)

# Create a heatmap using the Seaborn library
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, cmap="Blues", fmt='g', xticklabels=['Edible', 'Poisonous'], yticklabels=['Edible', 'Poisonous'])

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Test Confusion Matrix')

plt.show()
